<a href="https://colab.research.google.com/github/jyryu3161/lec_bioai/blob/main/featurization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
file_uploaded = files.upload()

Saving 4.herg_data.csv to 4.herg_data.csv


## 환경 구성

In [3]:
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

!pip install --pre deepchem

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3903    0  3903    0     0  52743      0 --:--:-- --:--:-- --:--:-- 52743
100 20.2M  100 20.2M    0     0  7887k      0  0:00:02  0:00:02 --:--:-- 8089k


## molecular descriptor 계산

In [74]:
import pandas as pd
import numpy as np
import tqdm

featurizer = dc.feat.RDKitDescriptors()

X_list = []
y_list = []
smiles_list = []

df = pd.read_csv('./4.herg_data.csv', sep=',') # 데이터 입력
for idx, each_df in tqdm.tqdm(df.iterrows()):
    smiles = each_df['smiles']
    Label = each_df['Label']

    features = featurizer.featurize(smiles) # 하나의 smiles에 대한 molecular descriptor 계산 
    descriptor_feature = features[0]
    
    if descriptor_feature.shape[0] != len(featurizer.descriptors): # molecular descriptor가 계산이 안되는 화합물은 고려하지 않음
        continue

    smiles_list.append(smiles)
    X_list.append(descriptor_feature)
    y_list.append(Label)

X_list = np.asarray(X_list)
y_list = np.asarray(y_list)
smiles_list = np.asarray(smiles_list)

df_tmp = pd.DataFrame(X_list, columns=featurizer.descriptors)
df_tmp['Label'] = y_list
df_tmp['Smiles'] = smiles_list
df_tmp.to_csv('./4.herg_data_descriptor.csv', index=False)


546it [00:06, 79.72it/s]WARNING:deepchem.feat.base_classes:Failed to featurize datapoint 0, None. Appending empty array
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
666it [00:08, 95.60it/s]WARNING:deepchem.feat.base_classes:Failed to featurize datapoint 0, None. Appending empty array
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
943it [00:11, 81.55it/s]


## molecular fingerprint 계산

In [73]:
import pandas as pd
import numpy as np
import tqdm

featurizer = dc.feat.CircularFingerprint(size=1024, radius=4)

X_list = []
y_list = []
smiles_list = []
cols_names = []
for i in range(1024):
    cols_names.append('x%s'%(i+1))

df = pd.read_csv('./4.herg_data.csv', sep=',') # 데이터 입력
for idx, each_df in tqdm.tqdm(df.iterrows()):
    smiles = each_df['smiles']
    Label = each_df['Label']

    features = featurizer.featurize(smiles) # 하나의 smiles에 대한 molecular fingerprint 계산 
    fingerprint_feature = features[0]

    if fingerprint_feature.shape[0] != 1024: # molecular descriptor가 계산이 안되는 화합물은 고려하지 않음
        continue

    smiles_list.append(smiles)
    X_list.append(fingerprint_feature)
    y_list.append(Label)

X_list = np.asarray(X_list)
y_list = np.asarray(y_list)
smiles_list = np.asarray(smiles_list)

df_tmp = pd.DataFrame(X_list, columns=cols_names)
df_tmp['Label'] = y_list
df_tmp['Smiles'] = smiles_list
df_tmp.to_csv('./4.herg_data_fingerprint.csv', index=False)


547it [00:00, 682.99it/s]WARNING:deepchem.feat.base_classes:Failed to featurize datapoint 0, None. Appending empty array
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
616it [00:00, 668.68it/s]WARNING:deepchem.feat.base_classes:Failed to featurize datapoint 0, None. Appending empty array
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
943it [00:01, 663.76it/s]
